In [ ]:
%run clean_data.ipynb
%run pitch_plot_fun.ipynb

In [ ]:
messi_grades = grades_df.query('`player.id` == "1531"')
clean_tracking_df = pd.read_csv('C:/Users/erden/OneDrive/Desktop/PFFFC/clean_tracking.csv')

In [ ]:
messi_grades[['possession_event_id', 'player.id']] = messi_grades[['possession_event_id', 'player.id']].astype(int)
messi_grades['playerGrade'] = messi_grades['playerGrade'].fillna(0)

In [ ]:
messi_grade_locs = messi_grades.merge(clean_tracking_df, on = ['possession_event_id', 'player.id', 'player.nickname'])
messi_grades_non_zero = messi_grade_locs.query('playerGrade != 0')

In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 7))

# Plot the pitch
plot_custom_pitch(ax=ax)


ax.plot([52.5, -52.5], [-20.16, -20.16], color='black', linestyle='--', linewidth=1)
ax.plot([52.5, -52.5], [-9.16, -9.16], color='black', linestyle='--', linewidth=1)
ax.plot([52.5, -52.5], [9.16, 9.16], color='black', linestyle='--', linewidth=1)
ax.plot([52.5, -52.5], [20.16, 20.16], color='black', linestyle='--', linewidth=1)

# Define custom colormap
colors = ['#5f03ff', '#0344ff', '#02a4df', '#97d5ec', '#ffcd00', '#df800e', '#f35b2b', '#ff0000']
cmap = ListedColormap(colors)

# Define the discrete grades as boundaries
grades = [-2, -1.5, -1, -0.5, 0.5, 1, 1.5, 2]
norm = BoundaryNorm(grades, len(colors))

# Scatter plot for Messi's grades with discrete color mapping
sc = ax.scatter(
    messi_grades_non_zero['x_normalized'], 
    messi_grades_non_zero['y_normalized'], 
    c=messi_grades_non_zero['playerGrade'], 
    s=100, 
    cmap=cmap, 
    norm=norm, 
    edgecolors='black'
)

# Custom legend mapping the exact grades to colors
legend_labels = ["2", "1.5", "1", "0.5", "-0.5", "-1", "-1.5", "-2"]
reversed_colors = list(reversed(colors))
legend_patches = [patches.Patch(color=reversed_colors[i], label=legend_labels[i]) for i in range(len(reversed_colors))]

# Add legend outside the field bounds
plt.legend(
    handles=legend_patches,
    title="Player Grade",
    loc='center left',
    bbox_to_anchor=(1.05, 0.5),  # Position the legend to the right
    fontsize=8,
    title_fontsize=10,
    frameon=False
)

plt.title('Messi PFF Grades by Field Zone')
plt.show()

In [ ]:
#Look at bunch of other stuff about Messi's grades (positive graded percentage based on different variables)
messi_grades['possession_event_id'] = messi_grades['possession_event_id'].astype(str)
messi_grades_detail = messi_grades.merge(possession_df, left_on ='possession_event_id', right_on = 'id')

In [ ]:
messi_grades_pa = messi_grades_detail.query('possessionEventType == "PA"')
messi_grades_pa['gradeType'] = messi_grades_pa['playerGrade'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Zero')

# Group by passingFoot and gradeType, then count
summary = messi_grades_pa.query('`passingEvent.passBodyType` in ["L", "R"]').groupby(['passingEvent.passBodyType', 'gradeType']).size().unstack(fill_value=0)

percentages = summary.div(summary.sum(axis=1), axis=0) * 100

# Plot a stacked bar chart
ax = percentages.plot(kind='bar', stacked=True, figsize=(8, 6), color=['red', 'green', 'white'],  edgecolor='black')

# Add labels and title
plt.title('Percentage of Positively vs Negatively Graded Plays by Passing Foot')
plt.ylabel('Percentage')
plt.xlabel('Passing Foot')
plt.legend(title='Grade Type', loc='upper right')
plt.xticks(rotation=0)

# Show values on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center')

plt.tight_layout()
plt.show()

In [ ]:
summary = messi_grades_pa.assign(pressureType=lambda x: np.where(x['passingEvent.pressureType'].isna(), 'NO', x['passingEvent.pressureType'])).groupby(['pressureType', 'gradeType']).size().unstack(fill_value=0)

percentages = summary.div(summary.sum(axis=1), axis=0) * 100

# Plot a stacked bar chart
ax = percentages.plot(kind='bar', stacked=True, figsize=(8, 6), color=['red', 'green', 'white'],  edgecolor='black')

# Add labels and title
plt.title('Percentage of Positively vs Negatively Graded Plays by Pressure Type')
plt.ylabel('Percentage')
plt.xlabel('Pressure Type')
plt.legend(title='Grade Type', loc='upper right')
plt.xticks(rotation=0)

# Show values on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center')

plt.tight_layout()
plt.show()

In [ ]:
messi_grades_sh = messi_grades_detail.query('possessionEventType == "SH"')
messi_grades_sh['gradeType'] = messi_grades_sh['playerGrade'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Zero')

# Group by passingFoot and gradeType, then count
summary = messi_grades_sh.query('`shootingEvent.shotBodyType` in ["L", "R"]').groupby(['shootingEvent.shotBodyType', 'gradeType']).size().unstack(fill_value=0)

percentages = summary.div(summary.sum(axis=1), axis=0) * 100

# Plot a stacked bar chart
ax = percentages.plot(kind='bar', stacked=True, figsize=(8, 6), color=['red', 'green', 'white'],  edgecolor='black')

# Add labels and title
plt.title('Percentage of Positively vs Negatively Graded Plays by Shooting Foot')
plt.ylabel('Percentage')
plt.xlabel('Shooting Foot')
plt.legend(title='Grade Type', loc='upper right')
plt.xticks(rotation=0)

# Show values on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center')

plt.tight_layout()
plt.show()

In [ ]:
summary = messi_grades_sh.assign(pressureType=lambda x: np.where(x['shootingEvent.pressureType'].isna(), 'NO', x['shootingEvent.pressureType'])).groupby(['pressureType', 'gradeType']).size().unstack(fill_value=0)

percentages = summary.div(summary.sum(axis=1), axis=0) * 100

# Plot a stacked bar chart
ax = percentages.plot(kind='bar', stacked=True, figsize=(8, 6), color=['red', 'green', 'white'],  edgecolor='black')

# Add labels and title
plt.title('Percentage of Positively vs Negatively Graded Plays by Pressure Type')
plt.ylabel('Percentage')
plt.xlabel('Pressure Type')
plt.legend(title='Grade Type', loc='upper right')
plt.xticks(rotation=0)

# Show values on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center')

plt.tight_layout()
plt.show()